In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Load the data
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')

In [2]:
# Merge transactions with customers and products
data = pd.merge(transactions, customers, on='CustomerID')
data = pd.merge(data, products, on='ProductID')

In [3]:
data

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CustomerName,Region,SignupDate,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 7:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,300.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,T00496,C0118,P037,2024-10-24 8:30:27,1,459.86,459.86,Jacob Holt,South America,2022-01-22,SoundWave Smartwatch,Electronics,459.86
996,T00759,C0059,P037,2024-06-04 2:15:24,3,1379.58,459.86,Mrs. Kimberly Wright,North America,2024-04-07,SoundWave Smartwatch,Electronics,459.86
997,T00922,C0018,P037,2024-04-05 13:05:32,4,1839.44,459.86,Tyler Haynes,North America,2024-09-21,SoundWave Smartwatch,Electronics,459.86
998,T00959,C0115,P037,2024-09-29 10:16:02,2,919.72,459.86,Joshua Hamilton,Asia,2024-11-11,SoundWave Smartwatch,Electronics,459.86


In [12]:
# Feature Engineering
# Create customer features based on their transaction history
customer_features = data.groupby('CustomerID').agg({
    'TotalValue': ['sum', 'mean', 'count'],
    'Quantity': ['sum', 'mean'],
    'Price_x': ['mean', 'std'],
    'Category': lambda x: x.mode()[0] if not x.mode().empty else 'Unknown'  # Most frequent category
}).reset_index()

In [13]:
customer_features

CustomerID TotalValue                    Quantity               Price_x  \
                      sum         mean count      sum      mean        mean   
0        C0001    3354.52   670.904000     5       12  2.400000  278.334000   
1        C0002    1862.74   465.685000     4       10  2.500000  208.920000   
2        C0003    2725.38   681.345000     4       14  3.500000  195.707500   
3        C0004    5354.88   669.360000     8       23  2.875000  240.636250   
4        C0005    2034.24   678.080000     3        7  2.333333  291.603333   
..         ...        ...          ...   ...      ...       ...         ...   
194      C0196    4982.88  1245.720000     4       12  3.000000  416.992500   
195      C0197    1928.65   642.883333     3        9  3.000000  227.056667   
196      C0198     931.83   465.915000     2        3  1.500000  239.705000   
197      C0199    1979.28   494.820000     4        9  2.250000  250.610000   
198      C0200    4758.60   951.720000     5       16  3.200000  296.506000   

                    Category  
            std     <lambda>  
0    176.857028  Electronics  
1    130.312477     Clothing  
2    145.351095   Home Decor  
3     80.751961        Books  
4    135.629902  Electronics  
..          ...          ...  
194   16.560206   Home Decor  
195  111.868454  Electronics  
196  300.824438     Clothing  
197  133.998588  Electronics  
198  130.093246     Clothing  

[199 rows x 9 columns]

In [14]:
# Flatten the multi-level columns
customer_features.columns = ['CustomerID', 'TotalValue_sum', 'TotalValue_mean', 'Transaction_count', 
                             'Quantity_sum', 'Quantity_mean', 'Price_mean', 'Price_std', 'Favorite_Category']

In [15]:
# Merge with customer information
customer_features = pd.merge(customer_features, customers, on='CustomerID')

In [16]:
# Handle missing values in numerical columns (replace NaN with 0 or mean)
customer_features['Price_std'].fillna(0, inplace=True)  # Replace NaN in Price_std with 0
customer_features['Price_mean'].fillna(customer_features['Price_mean'].mean(), inplace=True)  # Replace NaN in Price_mean with the mean

C:\Users\chint\AppData\Local\Temp\ipykernel_27516\861142852.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customer_features['Price_std'].fillna(0, inplace=True)  # Replace NaN in Price_std with 0
C:\Users\chint\AppData\Local\Temp\ipykernel_27516\861142852.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting va

In [17]:
# Encode categorical variables
customer_features = pd.get_dummies(customer_features, columns=['Region', 'Favorite_Category'])

In [18]:
customer_features

,CustomerID,TotalValue_sum,TotalValue_mean,Transaction_count,Quantity_sum,Quantity_mean,Price_mean,Price_std,CustomerName,SignupDate,Region_Asia,Region_Europe,Region_North America,Region_South America,Favorite_Category_Books,Favorite_Category_Clothing,Favorite_Category_Electronics,Favorite_Category_Home Decor
0,C0001,3354.52,670.904000,5,12,2.400000,278.334000,176.857028,Lawrence Carroll,2022-07-10,False,False,False,True,False,False,True,False
1,C0002,1862.74,465.685000,4,10,2.500000,208.920000,130.312477,Elizabeth Lutz,2022-02-13,True,False,False,False,False,True,False,False
2,C0003,2725.38,681.345000,4,14,3.500000,195.707500,145.351095,Michael Rivera,2024-03-07,False,False,False,True,False,False,False,True
3,C0004,5354.88,669.360000,8,23,2.875000,240.636250,80.751961,Kathleen Rodriguez,2022-10-09,False,False,False,True,True,False,False,False
4,C0005,2034.24,678.080000,3,7,2.333333,291.603333,135.629902,Laura Weber,2022-08-15,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,C0196,4982.88,1245.720000,4,12,3.000000,416.992500,16.560206,Laura Watts,2022-06-07,False,True,False,False,False,False,False,True
195,C0197,1928.65,642.883333,3,9,3.000000,227.056667,111.868454,Christina Harvey,2023-03-21,False,True,False,False,False,False,True,False
196,C0198,931.83,465.915000,2,3,1.500000,239.705000,300.824438,Rebecca Ray,2022-02-27,False,True,False,False,False,True,False,False
197,C0199,1979.28,494.820000,4,9,2.250000,250.610000,133.998588,Andrea Jenkins,2022-12-03,False,True,False,False,False,False,True,False


In [19]:
# Normalize the features
scaler = StandardScaler()
customer_features_scaled = scaler.fit_transform(customer_features.drop(columns=['CustomerID', 'CustomerName', 'SignupDate']))

In [20]:
# Calculate similarity matrix
similarity_matrix = cosine_similarity(customer_features_scaled)

In [21]:
similarity_matrix

array([[ 1.        , -0.24145128,  0.17668537, ...,  0.05414035,
         0.35660818, -0.28665678],
       [-0.24145128,  1.        , -0.09304936, ...,  0.38991937,
        -0.02039312,  0.55234877],
       [ 0.17668537, -0.09304936,  1.        , ..., -0.17745447,
        -0.22777504, -0.08489897],
       ...,
       [ 0.05414035,  0.38991937, -0.17745447, ...,  1.        ,
         0.43168058, -0.09954946],
       [ 0.35660818, -0.02039312, -0.22777504, ...,  0.43168058,
         1.        , -0.42940955],
       [-0.28665678,  0.55234877, -0.08489897, ..., -0.09954946,
        -0.42940955,  1.        ]])

In [22]:
# Create a DataFrame for the similarity matrix
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])

In [23]:
similarity_df

CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.000000,-0.241451,0.176685,0.058585,0.351358,0.167761,0.223586,-0.170520,-0.094092,-0.215521,...,0.276531,0.904680,-0.182859,-0.263319,0.155436,-0.316200,0.269135,0.054140,0.356608,-0.286657
C0002,-0.241451,1.000000,-0.093049,-0.373000,0.398248,-0.464782,0.221708,-0.187984,0.407340,0.517899,...,-0.245983,-0.068377,0.330175,-0.230816,-0.264101,-0.416701,-0.030222,0.389919,-0.020393,0.552349
C0003,0.176685,-0.093049,1.000000,0.188891,-0.236260,0.203399,-0.244142,0.104364,-0.396150,0.106072,...,0.080288,0.072073,-0.116367,-0.397735,0.837007,0.098773,0.007987,-0.177454,-0.227775,-0.084899
C0004,0.058585,-0.373000,0.188891,1.000000,-0.556393,0.491407,-0.406820,0.134515,-0.568528,-0.269096,...,0.520804,-0.180273,-0.071515,0.232727,0.436359,-0.091373,-0.397112,-0.633289,-0.466552,-0.057354
C0005,0.351358,0.398248,-0.236260,-0.556393,1.000000,-0.245323,0.885525,-0.389506,0.125777,-0.144320,...,-0.230869,0.471320,0.376398,-0.340864,-0.401989,-0.156106,0.410999,0.125086,0.446146,0.161202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C0196,-0.316200,-0.416701,0.098773,-0.091373,-0.156106,0.250657,0.212264,-0.129968,-0.145579,-0.219794,...,-0.318525,-0.399864,-0.301438,-0.422980,0.222956,1.000000,0.126044,-0.322599,-0.046974,0.133793
C0197,0.269135,-0.030222,0.007987,-0.397112,0.410999,-0.257725,0.364736,-0.409611,0.206645,0.454049,...,-0.337235,0.311708,-0.183129,-0.418880,-0.259637,0.126044,1.000000,0.225347,0.881642,-0.235384
C0198,0.054140,0.389919,-0.177454,-0.633289,0.125086,-0.351166,-0.200215,-0.160660,0.803150,0.600162,...,-0.039557,0.285877,0.120351,-0.075453,-0.406366,-0.322599,0.225347,1.000000,0.431681,-0.099549


In [24]:
# Function to get top 3 similar customers
def get_top_similar_customers(customer_id, similarity_df, top_n=3):
    similar_customers = similarity_df[customer_id].sort_values(ascending=False).iloc[1:top_n+1]
    return similar_customers.index.tolist(), similar_customers.values.tolist()

In [25]:
# Generate recommendations for the first 20 customers
lookalike_map = {}
for customer_id in customer_features['CustomerID'][:20]:
    similar_customers, scores = get_top_similar_customers(customer_id, similarity_df)
    lookalike_map[customer_id] = list(zip(similar_customers, scores))

In [26]:
# Convert the map to a DataFrame
lookalike_df = pd.DataFrame(list(lookalike_map.items()), columns=['CustomerID', 'Lookalikes'])

In [27]:
lookalike_df

,CustomerID,Lookalikes
0,C0001,"[(C0181, 0.9591702814458859), (C0190, 0.919755..."
1,C0002,"[(C0088, 0.9397339129691135), (C0106, 0.934231..."
2,C0003,"[(C0195, 0.8370071173477016), (C0052, 0.816491..."
3,C0004,"[(C0087, 0.8973897095457801), (C0153, 0.854803..."
4,C0005,"[(C0186, 0.9833343750031277), (C0146, 0.897907..."
5,C0006,"[(C0168, 0.9199353817753253), (C0171, 0.887538..."
6,C0007,"[(C0115, 0.9707057465354999), (C0146, 0.905739..."
7,C0008,"[(C0065, 0.8109809874346099), (C0189, 0.734517..."
8,C0009,"[(C0103, 0.9436548748417978), (C0198, 0.803150..."
9,C0010,"[(C0111, 0.9617279106896874), (C0062, 0.927605..."
